<a href="https://colab.research.google.com/github/Jih00nJung/assignment_list/blob/main/FashionGAN_v1-9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 파라미터
### v1
결과: 다양성은 30%정도 된 것같은데 형체가 무너짐, 원본 형태 강제가 더 필요할 듯
```
iter: 10000
sty: 5
ds: 10
cyc: 1
r1: 1
d_repeats: 2
```

### v2
결과: 원본의 강제성이 너무 심함 (500iter부터 드러남), 그리고 진행할 수록 원인불명의 뭉개짐과 흐려짐(1500~) 발생
```
iter: 5000
sty: 8
ds: 13
cyc: 2
r1: 1
d_repeats: 2
```

## v3
결과: v2때보단 덜 하지만 1500iter부터 점점 흐려짐과 뭉개짐 발생, 2 > 3 >>> 1이니까 파라미터 중 sty가 유력함(5 -> 8 -> 6), 일단 sty는 5로 고정하고 다른 변수 조정
```
iter: 2500
sty: 6
ds: 13
cyc: 1
r1: 1
d_repeats: 2
```

## v4
결과: 2500iter까지는 경과를 더 볼까 싶었는데 3000iter부터 급격히 뭉개짐과 흐려짐, ds에서 문제가 발생한건가봄, ds를 좀 줄이고 시도해보기
```
iter: 3500
sty: 5
ds: 12
cyc: 0.8
r1: 1
d_repeats: 2
```

## v5
결과: ds가 감소해서 다양성이 사라지고 3000iter부터 다시 흐려졌다. v1이랑 거의 비슷한 파라미터인데 왜 이런 결과인지 모르겠다.
```
iter: 4000
sty: 5
ds: 9
cyc: 0.8
r1: 1
d_repeats: 2
```

## v6
결과: 원본의 뭉개짐 문제가 거의 처음부터 발생했다. 판별자재호출을 다시 2로 늘려서 다시 시도해보자
```
iter: 5000
sty: 5
ds: 10
cyc: 1
r1: 0.8
d_repeats: 1
```

## v7
결과: v1의 경우를 봐서 판별자재호출을 3으로 늘려 시도해보자, 흐려짐은 나아졌다. (아마 판별자재호출의 영향인듯?) 하지만 형태가 뭉개졌다. cyc를 소폭 늘려보자
```
iter: 3500
sty: 5
ds: 10
cyc: 1
r1: 0.8
d_repeats: 2
```

## v5re (b1)
결과: 이때까지 한 것들 중 가장 괜찮아보여서 했는데 5500iter부터 뭉개지고 뿌옇게 됨
```
iter: 8000
sty: 5
ds: 9
cyc: 0.8
r1: 1
d_repeats: 2
batch: 64
```

## v7re (b2)
결과:
```
iter: 5000
sty: 5
ds: 10
cyc: 1
r1: 0.8
d_repeats: 2
batch: 64
```

## v8 (b3)
결과:
```
iter:
sty: 5
ds: 9
cyc: 0.01
r1: 1
d_repeats: 2
batch: 64
```

## v9 A100 (b4)
결과:
```
iter:
sty: 5
ds: 8
cyc: 1.2
r1: 1
d_repeats: 3
batch: 64
```

1. config.py (설정 및 환경 준비)

In [84]:
%%writefile config.py
import os
import argparse
from torchvision import transforms
from google.colab import drive

def get_config():
    """학습에 필요한 모든 하이퍼파라미터를 정의합니다."""
    parser = argparse.ArgumentParser()

    # 데이터 및 경로 설정
    parser.add_argument('--project_name', type=str, default='FashionGAN_v1-9')
    parser.add_argument('--save_root', type=str, default='/content/drive/MyDrive/Colab Notebooks/GAN_assignment')
    parser.add_argument('--img_size', type=int, default=64, help='이미지 크기 (FMNIST 기본 28 -> 64 리사이즈)')
    parser.add_argument('--batch_size', type=int, default=64)

    # 모델 하이퍼파라미터
    parser.add_argument('--style_dim', type=int, default=64, help='스타일 코드 차원')
    parser.add_argument('--latent_dim', type=int, default=16, help='랜덤 노이즈 차원')
    parser.add_argument('--num_domains', type=int, default=10, help='Fashion MNIST 클래스 개수')
    parser.add_argument('--hidden_dim', type=int, default=256, help='Mapping Network 히든 차원')

    # 학습 설정
    parser.add_argument('--total_iters', type=int, default=100000)
    parser.add_argument('--resume_iter', type=int, default=0)
    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--w_hpf', type=float, default=1, help='High-pass filtering 가중치')

    # Loss 가중치
    parser.add_argument('--lambda_sty', type=float, default=5.0)
    parser.add_argument('--lambda_ds', type=float, default=8.0)
    parser.add_argument('--lambda_cyc', type=float, default=1.2)
    parser.add_argument('--lambda_r1', type=float, default=1.0, help='R1 regularization loss 가중치') # R1 loss 가중치

    parser.add_argument('--d_train_repeats', type=int, default=3, help='Discriminator 학습 반복 횟수')

    # 로깅 주기
    parser.add_argument('--sample_freq', type=int, default=500) # 샘플 저장 주기
    parser.add_argument('--save_freq', type=int, default=5000)  # 체크포인트 및 평가 주기

    # FID/LPIPS 평가용 설정
    parser.add_argument('--num_fid_samples', type=int, default=1000, help='FID 계산에 사용할 생성 이미지 수')

    args, _ = parser.parse_known_args()
    return args

def prepare_environment(args):
    """Google Drive 마운트 및 체크포인트/샘플 디렉토리를 생성합니다."""
    print("--- 환경 설정 중 ---")
    save_path = os.path.join(args.save_root, args.project_name)

    # Google Drive 마운트 로직 제거 (외부 셀에서 마운트할 것임)
    if not os.path.exists('/content/drive'):

        print("Google Drive 마운트가 필요합니다. 학습 셀 실행 전에 Drive를 마운트해주세요.")

    os.makedirs(os.path.join(save_path, 'checkpoints'), exist_ok=True)
    os.makedirs(os.path.join(save_path, 'samples'), exist_ok=True)
    print(f"저장 경로: {save_path}")
    return save_path

def get_data_transform(img_size):
    """Fashion MNIST 데이터 전처리를 정의합니다."""
    return transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)) # [0,1] -> [-1,1]
    ])

def get_domain_labels():
    """Fashion MNIST의 10개 도메인 레이블을 반환합니다."""
    return ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
            'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Writing config.py


2. model.py (네트워크 아키텍처)

In [85]:
%%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdaIN(nn.Module):
    """Adaptive Instance Normalization"""
    def __init__(self, style_dim, num_features):
        super().__init__()
        # 1. 정규화 도구 (학습 파라미터 없음, 단순 통계 정규화)
        self.norm = nn.InstanceNorm2d(num_features, affine=False)
        # 2. 스타일 코드 s를 변환하여 감마(스케일)와 베타(시프트)를 만드는 선형 층
        self.fc = nn.Linear(style_dim, num_features * 2)

    def forward(self, x, s):
        # s를 통해 파라미터 생성 (h)
        h = self.fc(s)
        h = h.view(h.size(0), h.size(1), 1, 1)
        # 생성된 파라미터를 감마와 베타로 나눔
        gamma, beta = torch.chunk(h, chunks=2, dim=1)
        # 정규화된 x에 감마를 곱하고 베타를 더함 -> 스타일 주입
        return (1 + gamma) * self.norm(x) + beta

class ResBlock(nn.Module):
    """기본 ResBlock (다운샘플링 블록에서 사용)"""
    def __init__(self, dim_in, dim_out, actv=nn.LeakyReLU(0.2)):
        super().__init__()
        self.main = nn.Sequential(
            actv,
            nn.Conv2d(dim_in, dim_out, 3, 1, 1),
            nn.InstanceNorm2d(dim_out, affine=True),
            actv,
            nn.Conv2d(dim_out, dim_out, 3, 1, 1),
            nn.InstanceNorm2d(dim_out, affine=True)
        )
        self.shortcut = nn.Sequential()
        if dim_in != dim_out:
            self.shortcut = nn.Conv2d(dim_in, dim_out, 1, 1, 0)

    def forward(self, x):
        return self.main(x) + self.shortcut(x)

class AdaINResBlock(nn.Module):
    """Generator용 AdaIN ResBlock (Bottleneck 및 Up-sampling 블록에서 사용)"""
    def __init__(self, dim_in, dim_out, style_dim, actv=nn.LeakyReLU(0.2)):
        super().__init__()
        self.actv = actv
        self.conv1 = nn.Conv2d(dim_in, dim_out, 3, 1, 1)
        self.conv2 = nn.Conv2d(dim_out, dim_out, 3, 1, 1)
        self.norm1 = AdaIN(style_dim, dim_in)
        self.norm2 = AdaIN(style_dim, dim_out)

        self.shortcut = nn.Sequential()
        if dim_in != dim_out:
            self.shortcut = nn.Conv2d(dim_in, dim_out, 1, 1, 0)

    def forward(self, x, s):
        out = self.norm1(x, s)
        out = self.actv(out)
        out = self.conv1(out)
        out = self.norm2(out, s)
        out = self.actv(out)
        out = self.conv2(out)
        return out + self.shortcut(x)

# --- (1) Generator (G) ---
class Generator(nn.Module):
    def __init__(self, img_size=64, style_dim=64, max_conv_dim=512):
        super().__init__()
        dim_in = 64  # 경량화를 위한 시작 필터 수
        self.img_size = img_size

        # 1. 입력부: 흑백(1채널) 이미지를 받아서 32채널 특징 맵으로 변환
        self.from_rgb = nn.Conv2d(1, dim_in, 3, 1, 1) # Grayscale 1채널 입력

        # 2. 인코더 (Down-sampling): 형태 정보 압축
        # Down-sampling blocks (64 -> 8)
        self.encode = nn.ModuleList()
        curr_dim = dim_in
        for _ in range(3): # 8x8 병목 생성
            self.encode.append(ResBlock(curr_dim, curr_dim * 2))
            self.encode.append(nn.AvgPool2d(2))
            curr_dim = curr_dim * 2

        # 3. 병목 (Bottleneck): 스타일 주입 시작
        # Bottleneck (8x8 유지, AdaIN 적용)
        self.decode = nn.ModuleList()
        curr_dim = min(curr_dim, max_conv_dim)
        for _ in range(2):
            self.decode.append(AdaINResBlock(curr_dim, curr_dim, style_dim))

        # 4. 디코더 (Up-sampling): 이미지 복원 + 스타일 입히기
        # Up-sampling blocks (8 -> 64)
        for _ in range(3):
            self.decode.append(nn.Upsample(scale_factor=2, mode='nearest'))
            self.decode.append(AdaINResBlock(curr_dim, curr_dim // 2, style_dim))
            curr_dim = curr_dim // 2

        # 5. 출력부: 최종적으로 1채널(흑백) 이미지로 변환
        # Final Conv
        self.to_rgb = nn.Sequential(
            nn.InstanceNorm2d(curr_dim, affine=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(curr_dim, 1, 1, 1, 0) # Grayscale 1채널 출력
        )

    def forward(self, x, s):
        x = self.from_rgb(x)
        for block in self.encode:
            x = block(x)

        for block in self.decode:
            if isinstance(block, AdaINResBlock):
                x = block(x, s)
            else:
                x = block(x)

        return self.to_rgb(x)

# --- (2) Mapping Network (F) ---
class MappingNetwork(nn.Module):
    def __init__(self, latent_dim=16, style_dim=64, num_domains=10, hidden_dim=256):
        super().__init__()
        layers = []

        # 공유 레이어 (Shared)
        for _ in range(3):
            layers += [nn.Linear(latent_dim if not layers else hidden_dim, hidden_dim)]
            layers += [nn.ReLU()]
        # 1. 공유 레이어 (Shared): 모든 도메인이 공통으로 사용하는 특징 추출
        self.shared = nn.Sequential(*layers)

        # 2. 비공유 레이어 (Unshared): 각 도메인(T-shirt, Pants...)별 전용 스타일 생성기
        # 도메인별 출력 레이어 (Unshared)
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared.append(nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, style_dim)
            ))

    def forward(self, z, y):
        h = self.shared(z)
        out = []
        for layer in self.unshared:
            out += [layer(h)]
        out = torch.stack(out, dim=1) # (batch, num_domains, style_dim)

        # 사용자가 요청한 도메인(y)에 해당하는 스타일만 쏙 뽑아서 리턴
        idx = torch.arange(y.size(0)).to(y.device)
        s = out[idx, y] # 해당 도메인의 스타일 코드만 선택
        return s

# --- (3) Style Encoder (E) ---
class StyleEncoder(nn.Module):
    def __init__(self, img_size=64, style_dim=64, num_domains=10):
        super().__init__()
        dim_in = 32
        blocks = []
        blocks += [nn.Conv2d(1, dim_in, 3, 1, 1)] # 1ch input

        curr_dim = dim_in
        # Downsample to small size (64 -> 8)
        for _ in range(3): # 64 -> 8
            blocks += [nn.LeakyReLU(0.2)]
            blocks += [nn.Conv2d(curr_dim, curr_dim * 2, 3, 2, 1)]
            curr_dim = curr_dim * 2

        blocks += [nn.LeakyReLU(0.2)]
        blocks += [nn.Conv2d(curr_dim, curr_dim, 8, 1, 0)] # 8x8 -> 1x1
        # 1. 공유 레이어: 이미지를 보며 특징을 추출 (CNN 구조)
        # 64 -> 32 -> 16 -> 8 로 줄어들며 추상적인 특징을 잡아냄
        self.shared = nn.Sequential(*blocks)

        # 2. 비공유 레이어: 추출된 특징을 보고 "이건 바지 스타일로는 s_pants, 티셔츠로는 s_shirt야" 라고 해석
        # 도메인별 Style Code 출력
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared.append(nn.Linear(curr_dim, style_dim))

    def forward(self, x, y):
        # 이미지 x에서 시각적 특징 추출
        h = self.shared(x) # (batch, curr_dim, 1, 1)
        h = h.view(h.size(0), -1)

        out = []
        for layer in self.unshared:
            out += [layer(h)]
        out = torch.stack(out, dim=1)

        idx = torch.arange(y.size(0)).to(y.device)
        s = out[idx, y]
        return s

# --- (4) Discriminator (D) ---
class Discriminator(nn.Module):
    def __init__(self, img_size=64, num_domains=10):
        super().__init__()
        dim_in = 32
        blocks = []
        blocks += [nn.Conv2d(1, dim_in, 3, 1, 1)] # 1ch input

        curr_dim = dim_in
        # Downsample to small size (64 -> 8)
        for _ in range(3): # 64 -> 8
            blocks += [nn.LeakyReLU(0.2)]
            blocks += [nn.Conv2d(curr_dim, curr_dim * 2, 3, 2, 1)]
            curr_dim = curr_dim * 2

        blocks += [nn.LeakyReLU(0.2)]
        blocks += [nn.Conv2d(curr_dim, curr_dim, 8, 1, 0)] # 8x8 -> 1x1
        # 1. 공유 레이어: 이미지가 진짜인지 가짜인지 판단하기 위한 단서(특징) 추출
        # ResBlock이나 Conv 레이어를 사용하여 이미지를 분석함
        self.shared = nn.Sequential(*blocks)

        # 2. 멀티 태스크 헤드: 각 도메인별로 진위 여부를 따로 판별
        # 도메인별 진위 판별 헤드
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared.append(nn.Linear(curr_dim, 1))

    def forward(self, x, y):
        h = self.shared(x)
        h = h.view(h.size(0), -1)

        out = []
        for layer in self.unshared:
            out += [layer(h)]
        out = torch.stack(out, dim=1) # (batch, num_domains, 1)

        idx = torch.arange(y.size(0)).to(y.device)
        score = out[idx, y]
        return score

Writing model.py


3. FID와 LPIPS

In [86]:
!pip install torch-fidelity

In [87]:
%%writefile fid_lpips_calculator.py
import torch
import numpy as np
import os
import shutil
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch_fidelity import calculate_metrics

def generate_and_save_fake_images(G, F, args, device, samples_count, output_dir):
    """
    Fake 이미지를 samples_count만큼 생성하여 디스크에 개별 파일로 저장합니다.
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    generated_count = 0
    while generated_count < samples_count:
        current_batch_size = min(args.batch_size, samples_count - generated_count)

        with torch.no_grad():
            z_trg = torch.randn(current_batch_size, args.latent_dim).to(device)
            y_trg = torch.randint(0, args.num_domains, (current_batch_size,)).to(device)

            x_real_dummy = torch.zeros(
                current_batch_size, 1, args.img_size, args.img_size
            ).to(device)

            s_trg = F(z_trg, y_trg)
            x_fake = G(x_real_dummy, s_trg)

            # [-1, 1] 범위의 이미지를 [0, 1]로 변환하여 저장
            x_fake_normalized = (x_fake + 1) / 2

        # 이미지를 디스크에 개별 파일로 저장
        for i in range(current_batch_size):
            filename = os.path.join(output_dir, f'fake_{generated_count + i:04d}.png')
            save_image(x_fake_normalized[i], filename)

        generated_count += current_batch_size

    return output_dir

def prepare_real_images(args, output_dir):
    """
    Fashion MNIST 테스트 셋을 로드하여 PNG 이미지 파일로 변환 후 디스크에 저장합니다.
    """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    to_tensor = transforms.Compose([
        transforms.Resize((args.img_size, args.img_size)),
        transforms.ToTensor()
    ])

    test_dataset = datasets.FashionMNIST(root='/content/data', train=False, download=False, transform=to_tensor)

    for i, (img_tensor, _) in enumerate(test_dataset):
        if i >= 10000:
             break

        filename = os.path.join(output_dir, f'real_{i:04d}.png')
        save_image(img_tensor, filename)

    return output_dir

def compute_fid(G, F, args, device, num_gen=1000): # 함수명을 compute_fid로 변경
    """
    저장된 Real Data와 생성된 Fake Data를 비교하여 FID를 계산합니다.
    """

    FAKE_DIR = '/content/fid_tmp/fake'
    REAL_DIR = '/content/fid_tmp/real'

    # 1. Real 이미지 디스크 저장 (최초 1회만 필요)
    if not os.path.exists(REAL_DIR) or len(os.listdir(REAL_DIR)) < 10000:
        print(f"Real 이미지 10000개 디스크 준비 중...")
        prepare_real_images(args, REAL_DIR)

    # 2. Fake 이미지 생성 및 디스크 저장
    print(f"FID 계산을 시작합니다. (생성 이미지 {num_gen}개 디스크 저장 중...)")
    generate_and_save_fake_images(G, F, args, device, num_gen, FAKE_DIR)

    # 3. FID 계산 실행
    print("FID 계산을 시작합니다. (Inception V3 모델 로딩 중...)")

    metrics = calculate_metrics(
        input1=REAL_DIR,
        input1_name='real',

        input2=FAKE_DIR,
        input2_name='generated',

        cuda=device.type == 'cuda',
        kid=True,
        fid=True,
        lpips=False, # LPIPS 계산 제외
        verbose=False,
    )

    # 임시 디렉터리 삭제
    shutil.rmtree(FAKE_DIR)

    # FID 점수만 반환
    fid_score = metrics.get('frechet_inception_distance', float('inf'))

    return fid_score

Writing fid_lpips_calculator.py


4. solver.py (메인 실행 및 학습 루프)

In [88]:
%%writefile solver.py
import os
import time
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.utils import save_image, make_grid
from torch.autograd import grad as torch_grad

# 분리된 파일에서 모듈 가져오기
from config import get_config, prepare_environment, get_data_transform, get_domain_labels
from model import Generator, MappingNetwork, StyleEncoder, Discriminator


class Solver:
    def __init__(self, args, device):
        self.args = args
        self.device = device
        self.save_dir = prepare_environment(args)
        self.domain_labels = get_domain_labels()

        # 데이터셋 준비 (FashionMNIST)
        transform = get_data_transform(args.img_size)

        # 학습용 데이터 로더 (안정화 설정 적용)
        dataset = datasets.FashionMNIST(root='/content/data', train=True, download=False, transform=transform)
        self.loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True, num_workers=12, drop_last=True, pin_memory=True)

        # 모델 초기화
        self.nets = {
            'G': Generator(args.img_size, args.style_dim),
            'F': MappingNetwork(args.latent_dim, args.style_dim, args.num_domains, args.hidden_dim),
            'E': StyleEncoder(args.img_size, args.style_dim, args.num_domains),
            'D': Discriminator(args.img_size, args.num_domains)
        }

        for name, module in self.nets.items():
            module.to(self.device)
            module.train()

        # 옵티마이저 (D의 betas=(0.0, 0.99)로 수정 완료)
        self.optims = {
            'G': torch.optim.Adam(self.nets['G'].parameters(), lr=args.lr, betas=(0.0, 0.99)),
            'F': torch.optim.Adam(self.nets['F'].parameters(), lr=args.lr*0.01, betas=(0.0, 0.99)),
            'E': torch.optim.Adam(self.nets['E'].parameters(), lr=args.lr, betas=(0.0, 0.99)),
            'D': torch.optim.Adam(self.nets['D'].parameters(), lr=args.lr, betas=(0.0, 0.99))
        }

        # 체크포인트 로드
        self.start_iter = 0
        if args.resume_iter > 0:
            self.load_checkpoint(args.resume_iter)
            self.start_iter = args.resume_iter

    def save_checkpoint(self, step):
        path = os.path.join(self.save_dir, 'checkpoints', f'{step:06d}.ckpt')
        state = {
            'nets': {name: net.state_dict() for name, net in self.nets.items()},
            'optims': {name: opt.state_dict() for name, opt in self.optims.items()},
            'step': step
        }
        torch.save(state, path)
        print(f"Saved checkpoint to {path}")

    def load_checkpoint(self, step):
        path = os.path.join(self.save_dir, 'checkpoints', f'{step:06d}.ckpt')
        if not os.path.exists(path):
            print("Checkpoint not found!")
            return

        ckpt = torch.load(path, map_location=self.device)
        for name, net in self.nets.items():
            net.load_state_dict(ckpt['nets'][name])
        for name, opt in self.optims.items():
            opt.load_state_dict(ckpt['optims'][name])
        print(f"Loaded checkpoint from {path}")

    def r1_loss(self, d_out, x_in):
        """Discriminator의 R1 Gradient Penalty를 계산합니다."""
        grad_dout = torch_grad(
            outputs=d_out.sum(), inputs=x_in,
            create_graph=True, retain_graph=True, only_inputs=True
        )[0]
        grad_dout2 = grad_dout.pow(2)
        assert(grad_dout2.size() == x_in.size())
        r1_loss = grad_dout2.reshape(x_in.size(0), -1).sum(1).mean(0)
        return r1_loss

    def train(self):
        print("--- 학습 시작 ---\n")
        nets = self.nets
        optims = self.optims
        args = self.args

        data_iter = iter(self.loader)

        start_time = time.time()
        for i in range(self.start_iter, args.total_iters):

            # D를 G보다 d_train_repeats 만큼 더 학습시킵니다.
            for d_repeat in range(args.d_train_repeats): # <-- D 반복 학습 루프 시작

                # 1. 데이터 가져오기
                try:
                    x_real, y_org = next(data_iter)
                except StopIteration:
                    data_iter = iter(self.loader)
                    x_real, y_org = next(data_iter)

                x_real = x_real.to(self.device)
                y_org = y_org.to(self.device)

                # R1 Loss 계산을 위해 x_real에 그래디언트 추적 활성화
                x_real.requires_grad_(True)

                # 타겟 도메인 및 Latent 생성
                y_trg = torch.randint(0, args.num_domains, (x_real.size(0),)).to(self.device)
                z_trg = torch.randn(x_real.size(0), args.latent_dim).to(self.device)

                # =================================================================================== #
                #                               1. Discriminator 학습                                 #
                # =================================================================================== #

                # Real Loss
                d_out_real = nets['D'](x_real, y_org)
                d_loss_real = torch.mean(F.relu(1.0 - d_out_real))

                # R1 Regularization Loss 계산
                d_loss_r1 = self.r1_loss(d_out_real, x_real)

                # Fake Loss (Latent 기반 생성)
                with torch.no_grad():
                    s_trg = nets['F'](z_trg, y_trg)
                    x_fake = nets['G'](x_real, s_trg)

                d_out_fake = nets['D'](x_fake.detach(), y_trg)
                d_loss_fake = torch.mean(F.relu(1.0 + d_out_fake))

                # D Total Loss: Hinge Loss + R1 Loss
                d_loss = d_loss_real + d_loss_fake + args.lambda_r1 * d_loss_r1

                optims['D'].zero_grad()
                d_loss.backward()
                optims['D'].step()

                # 그래디언트 추적 해제
                x_real.requires_grad_(False)

            # G 학습은 1번만 수행
            # =================================================================================== #
            #                     2. Generator, Mapping, Encoder 학습                             #
            # =================================================================================== #

            # G 학습에 사용할 z_trg, z_trg2는 여기서 생성
            z_trg = torch.randn(x_real.size(0), args.latent_dim).to(self.device)
            z_trg2 = torch.randn(x_real.size(0), args.latent_dim).to(self.device)

            # Adversarial Loss
            s_trg = nets['F'](z_trg, y_trg)
            x_fake = nets['G'](x_real, s_trg)
            d_out_fake = nets['D'](x_fake, y_trg)
            g_loss_adv = -torch.mean(d_out_fake)

            # Style Reconstruction Loss
            s_pred = nets['E'](x_fake, y_trg)
            g_loss_sty = torch.mean(torch.abs(s_trg - s_pred))

            # Diversity Sensitive Loss
            s_trg2 = nets['F'](z_trg2, y_trg)
            x_fake2 = nets['G'](x_real, s_trg2)
            g_loss_ds = torch.mean(torch.abs(x_fake - x_fake2))

            # Cycle Consistency Loss
            s_org = nets['E'](x_real, y_org)
            x_rec = nets['G'](x_fake, s_org)
            g_loss_cyc = torch.mean(torch.abs(x_real - x_rec))

            # Total Loss
            g_loss = g_loss_adv \
                     + args.lambda_sty * g_loss_sty \
                     - args.lambda_ds * g_loss_ds \
                     + args.lambda_cyc * g_loss_cyc

            optims['G'].zero_grad()
            optims['F'].zero_grad()
            optims['E'].zero_grad()
            g_loss.backward()
            optims['G'].step()
            optims['F'].step()
            optims['E'].step()

            # =================================================================================== #
            #                                 3. 로깅 및 저장                                     #
            # =================================================================================== #

            # 100 iter마다 로깅 및 평가 (save_freq 설정에 따름)
            if (i + 1) % 100 == 0:
                elapsed = time.time() - start_time
                print(f"Iter [{i+1}/{args.total_iters}] Time: {elapsed:.2f}s | "
                      f"D_loss: {d_loss.item():.4f} | G_adv: {g_loss_adv.item():.4f} | "
                      f"Sty: {g_loss_sty.item():.4f} | Cyc: {g_loss_cyc.item():.4f}")

            if (i + 1) % args.sample_freq == 0:
                self.save_samples(x_real, y_org, i + 1)

            if (i + 1) % args.save_freq == 0:
                self.save_checkpoint(i + 1)

    def save_samples(self, x_real, y_org, step):
        """학습 중간 결과 이미지 저장 (도메인 라벨 포함 시각화 개선)"""
        nets = self.nets
        args = self.args

        with torch.no_grad():
            nets['G'].eval()
            nets['F'].eval()

            x_real_subset = x_real[:args.num_domains].to(self.device)

            z_fix = torch.randn(1, args.latent_dim).repeat(args.num_domains, 1).to(self.device)
            y_fix = torch.arange(args.num_domains).to(self.device)
            s_fix = nets['F'](z_fix, y_fix)

            images = []

            # 1. 소스 이미지
            source_row = [x_real_subset[i].cpu() for i in range(len(x_real_subset))]
            images.extend(source_row)

            # 2. 나머지 영역: 변환된 이미지 (스타일 변환 매트릭스)
            for j in range(args.num_domains):
                s_curr = s_fix[j].unsqueeze(0).repeat(x_real_subset.size(0), 1)
                x_fake_row = nets['G'](x_real_subset, s_curr)
                images.extend([x_fake_row[i].cpu() for i in range(len(x_real_subset))])

            images = torch.stack(images, dim=0)

            path = os.path.join(self.save_dir, 'samples', f'{step:06d}_grid.jpg')
            save_image(images, path, nrow=len(x_real_subset), padding=2, normalize=True)
            print(f"Sample image grid saved to {path}")

        # 다시 학습 모드
        nets['G'].train()
        nets['F'].train()

if __name__ == '__main__':
    # 시드 고정
    torch.manual_seed(777)
    np.random.seed(777)

    # 설정 로드
    config = get_config()

    # 장치 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    # Solver 시작
    solver = Solver(config, device)
    solver.train()

Writing solver.py


학습 진행

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
import torchvision.datasets as datasets

datasets.FashionMNIST(root='/content/data', train=True, download=True)
datasets.FashionMNIST(root='/content/data', train=False, download=True)

print("Fashion MNIST 데이터셋 다운로드 및 준비 완료. 이제 코드에서 download=False로 설정 가능합니다.")

Fashion MNIST 데이터셋 다운로드 및 준비 완료. 이제 코드에서 download=False로 설정 가능합니다.


In [91]:
!python solver.py

Device: cuda
--- 환경 설정 중 ---
저장 경로: /content/drive/MyDrive/Colab Notebooks/GAN_assignment/FashionGAN_v1-9
--- 학습 시작 ---

Iter [100/100000] Time: 28.07s | D_loss: 0.4296 | G_adv: 1.5965 | Sty: 0.0094 | Cyc: 0.3910
Iter [200/100000] Time: 55.30s | D_loss: 0.3296 | G_adv: 1.9685 | Sty: 0.0090 | Cyc: 0.3576
Iter [300/100000] Time: 82.44s | D_loss: 0.4180 | G_adv: 1.7920 | Sty: 0.0085 | Cyc: 0.3347
Iter [400/100000] Time: 109.91s | D_loss: 0.3341 | G_adv: 2.0609 | Sty: 0.0100 | Cyc: 0.3322
Iter [500/100000] Time: 137.13s | D_loss: 0.8764 | G_adv: 1.7631 | Sty: 0.0088 | Cyc: 0.3196
Sample image grid saved to /content/drive/MyDrive/Colab Notebooks/GAN_assignment/FashionGAN_v1-9/samples/000500_grid.jpg
Iter [600/100000] Time: 164.46s | D_loss: 0.5457 | G_adv: 2.0876 | Sty: 0.0096 | Cyc: 0.3517
Iter [700/100000] Time: 192.01s | D_loss: 0.5308 | G_adv: 1.5064 | Sty: 0.0102 | Cyc: 0.3133
Iter [800/100000] Time: 219.17s | D_loss: 0.7943 | G_adv: 1.6826 | Sty: 0.0098 | Cyc: 0.3194
Iter [900/100000]

evaluate

In [109]:
%%writefile evaluate.py
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from fid_lpips_calculator import compute_fid

# 분리된 파일에서 모듈 가져오기
from config import get_config, get_data_transform, get_domain_labels
from model import Generator, MappingNetwork, StyleEncoder, Discriminator


def load_model(args, device, checkpoint_step):
    """지정된 체크포인트를 불러와 G, F, E 네트워크를 반환합니다."""
    model_path = os.path.join(args.save_root, args.project_name, 'checkpoints')
    latest_ckpt = f'{checkpoint_step:06d}.ckpt'

    ckpt_path = os.path.join(model_path, latest_ckpt)
    if not os.path.exists(ckpt_path):
        raise FileNotFoundError(f"체크포인트 파일이 없습니다: {ckpt_path}")

    print(f"\nLoading checkpoint: {ckpt_path}")

    ckpt = torch.load(ckpt_path, map_location=device)

    # 모델 초기화 및 가중치 로드
    nets = {
        'G': Generator(args.img_size, args.style_dim).to(device),
        'F': MappingNetwork(args.latent_dim, args.style_dim, args.num_domains, args.hidden_dim).to(device),
        'E': StyleEncoder(args.img_size, args.style_dim, args.num_domains).to(device)
    }

    for name, net in nets.items():
        net.load_state_dict(ckpt['nets'][name])
        net.eval()

    print(f"Models loaded at iteration {checkpoint_step}.")
    return nets['G'], nets['F'], nets['E']


def compute_metrics_and_visualize(args, device, checkpoint_step):
    """
    체크포인트 로드, FID 계산 및 이미지 시각화를 수행합니다.
    """

    # 1. 모델 로드
    G, F, E = load_model(args, device, checkpoint_step) # E(StyleEncoder)도 함께 로드할 수 있도록 수정

    # 2. FID 계산 (FID만 수행)
    fid_score = compute_fid(G, F, args, device, args.num_fid_samples)

    print("\n--- 평가 결과 ---")
    print(f"FID Score: {fid_score:.4f} (Lower is better)")
    print("-----------------\n")

    # 3. 시각화 준비 및 실행

    # 테스트 데이터 로드 (시각화용)
    transform = get_data_transform(args.img_size)
    test_dataset = datasets.FashionMNIST(root='/content/data', train=False, download=False, transform=transform)

    # 10장만 샘플링하기 위해 DataLoader 사용
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

    # 소스 이미지 가져오기
    x_real, y_org = next(iter(test_loader))
    x_real = x_real.to(device)

    domain_labels = get_domain_labels()

    plt.figure(figsize=(15, 10))

    # [Row 1] 원본 이미지 (Source)
    for i in range(10):
        plt.subplot(args.num_domains + 1, 10, i + 1)
        img = x_real[i].cpu().squeeze().numpy()
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        if i == 0: plt.title("Source", fontsize=12, loc='left')

    # [Rows 2-11] 각 도메인으로 스타일 변환 (Latent Guided)
    z_trg = torch.randn(1, args.latent_dim).to(device)

    for row_idx in range(args.num_domains):
        y_trg = torch.tensor([row_idx]).to(device)
        s_trg = F(z_trg, y_trg) # (1, style_dim)
        s_trg = s_trg.repeat(10, 1) # 스타일 코드를 배치 크기만큼 복사

        # 이미지 생성
        with torch.no_grad():
            x_fake = G(x_real, s_trg)

        # 결과 출력
        for col_idx in range(10):
            plt.subplot(args.num_domains + 1, 10, (row_idx + 1) * 10 + col_idx + 1)
            img = x_fake[col_idx].cpu().squeeze().numpy()
            plt.imshow(img, cmap='gray')
            plt.axis('off')

            if col_idx == 0:
                plt.text(-25, 32, domain_labels[row_idx], fontsize=10, va='center', ha='right', color='black')

    plt.tight_layout()
    plt.show()

# 실행
if __name__ == '__main__':
    # 테스트를 원하는 iteration 번호를 직접 지정하세요. (예: 100 iter)
    TEST_STEP = 20000

    config = get_config()

    # config.py의 project_name
    config.project_name = 'FashionGAN_v1-9'

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # FID 계산 및 시각화 실행
    compute_metrics_and_visualize(config, device, TEST_STEP)

Writing evaluate.py


In [112]:
%%writefile evaluate.py
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from fid_lpips_calculator import compute_fid
import re

# 분리된 파일에서 모듈 가져오기
from config import get_config, get_data_transform, get_domain_labels
from model import Generator, MappingNetwork, StyleEncoder, Discriminator


def load_model(args, device, checkpoint_step):
    """지정된 체크포인트를 불러와 G, F, E 네트워크를 반환합니다."""
    model_path = os.path.join(args.save_root, args.project_name, 'checkpoints')
    latest_ckpt = f'{checkpoint_step:06d}.ckpt'

    ckpt_path = os.path.join(model_path, latest_ckpt)
    if not os.path.exists(ckpt_path):
        raise FileNotFoundError(f"체크포인트 파일이 없습니다: {ckpt_path}")

    # 모델 초기화 (load_state_dict 전에 초기화 필요)
    nets = {
        'G': Generator(args.img_size, args.style_dim).to(device),
        'F': MappingNetwork(args.latent_dim, args.style_dim, args.num_domains, args.hidden_dim).to(device),
        'E': StyleEncoder(args.img_size, args.style_dim, args.num_domains).to(device)
    }

    ckpt = torch.load(ckpt_path, map_location=device)

    # 가중치 로드 및 eval 모드 설정
    for name, net in nets.items():
        net.load_state_dict(ckpt['nets'][name])
        net.eval()

    return nets['G'], nets['F'], nets['E']


def plot_fid_history(steps, fids, args):
    """주어진 FID 기록을 그래프로 시각화하고 저장합니다."""
    plt.figure(figsize=(10, 6))
    plt.plot(steps, fids, marker='o', linestyle='-', color='blue')
    plt.title(f'FID Score History ({args.project_name})', fontsize=15)
    plt.xlabel('Iteration Step', fontsize=12)
    plt.ylabel('FID Score (Lower is better)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)

    # --- FID 수치 Annotation 추가 로직 ---
    for step, fid in zip(steps, fids):
        # 수치를 표시할 위치 (x, y)와 텍스트 포맷을 지정
        plt.annotate(f'{fid:.2f}', (step, fid),
                     textcoords="offset points",
                     xytext=(0, 5), # 점 위로 5포인트 이동
                     ha='center',
                     fontsize=9,
                     color='darkred')

    plt.xticks(steps, rotation=45)
    plt.tight_layout()

    # 그래프 저장 로직 추가
    save_dir = os.path.join(args.save_root, args.project_name)
    os.makedirs(save_dir, exist_ok=True) # 저장 디렉토리 확인
    save_path = os.path.join(save_dir, f'fid_history_{args.project_name}.png')

    plt.savefig(save_path) # PNG 파일로 저장
    print(f"\nFID 그래프 저장 완료: {save_path}")

    plt.show() # Colab 노트북 셀에서 시각화 출력 시도


def get_checkpoint_steps(args, step_interval=5000):
    """
    체크포인트 폴더에서 5000 iter 간격의 .ckpt 파일을 찾아 목록을 반환합니다.
    """
    model_path = os.path.join(args.save_root, args.project_name, 'checkpoints')
    if not os.path.exists(model_path):
        print(f"경고: 체크포인트 디렉터리를 찾을 수 없습니다: {model_path}")
        return []

    ckpts = sorted([f for f in os.listdir(model_path) if f.endswith('.ckpt')])

    valid_steps = []

    for ckpt in ckpts:
        match = re.match(r'(\d{6})\.ckpt', ckpt)
        if match:
            step = int(match.group(1))
            if step > 0 and step % step_interval == 0:
                valid_steps.append(step)

    return sorted(list(set(valid_steps)))


def visualize_samples(args, device, G, F, step):
    """단일 체크포인트에 대한 이미지 그리드 시각화를 수행하고 저장합니다."""
    transform = get_data_transform(args.img_size)
    test_dataset = datasets.FashionMNIST(root='/content/data', train=False, download=False, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

    x_real, y_org = next(iter(test_loader))
    x_real = x_real.to(device)

    domain_labels = get_domain_labels()
    plt.figure(figsize=(15, 10))
    images_list = []

    # [Row 1] 원본 이미지 (Source)
    for i in range(10):
        plt.subplot(args.num_domains + 1, 10, i + 1)
        img = x_real[i].cpu().squeeze().numpy()
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        if i == 0: plt.title("Source", fontsize=12, loc='left')
        images_list.append(x_real[i].cpu())

    # [Rows 2-11] 각 도메인으로 스타일 변환 (Latent Guided)
    z_trg = torch.randn(1, args.latent_dim).to(device)

    for row_idx in range(args.num_domains):
        y_trg = torch.tensor([row_idx]).to(device)
        s_trg = F(z_trg, y_trg)
        s_trg = s_trg.repeat(10, 1)

        with torch.no_grad():
            x_fake = G(x_real, s_trg)

        for col_idx in range(10):
            plt.subplot(args.num_domains + 1, 10, (row_idx + 1) * 10 + col_idx + 1)
            img = x_fake[col_idx].cpu().squeeze().numpy()
            plt.imshow(img, cmap='gray')
            plt.axis('off')

            if col_idx == 0:
                plt.text(-25, 32, domain_labels[row_idx], fontsize=10, va='center', ha='right', color='black')

            images_list.append(x_fake[col_idx].cpu())

    plt.tight_layout()
    plt.show()

    # 이미지 파일로 저장하는 로직
    save_dir = os.path.join(args.save_root, args.project_name, 'eval_samples')
    os.makedirs(save_dir, exist_ok=True)

    grid_tensor = torch.stack(images_list, dim=0)
    save_path = os.path.join(save_dir, f'{step:06d}_eval_grid.png')

    save_image(grid_tensor, save_path, nrow=10, normalize=True)
    print(f"\n이미지 그리드 저장 완료: {save_path}")


def main():
    config = get_config()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    config.project_name = 'FashionGAN_v1-9'

    # --- 1. FID 기록 계산 ---
    steps_to_evaluate = get_checkpoint_steps(config, step_interval=5000)
    fid_results = {}

    print(f"--- 프로젝트: {config.project_name} ---")
    print(f"평가할 체크포인트 (5000 iter 간격): {steps_to_evaluate}")

    for step in steps_to_evaluate:
        print(f"\n[Evaluating Iteration {step}]")
        try:
            G, F, E = load_model(config, device, step)

            # FID 계산
            fid_score = compute_fid(G, F, config, device, config.num_fid_samples)

            fid_results[step] = fid_score
            print(f"FID Score @ {step}: {fid_score:.4f}")

        except FileNotFoundError as e:
            print(f"오류: {e}")
            continue
        except RuntimeError as e:
            print(f"런타임 오류: {e}")
            continue

    # --- 2. FID 추이 그래프 시각화 및 저장 ---

    if fid_results:
        steps = sorted(fid_results.keys())
        fids = [fid_results[s] for s in steps]

        # 그래프 출력 및 저장 (config를 인수로 전달)
        plot_fid_history(steps, fids, config)
    else:
        print("\n계산된 FID 기록이 없어 그래프를 생성할 수 없습니다.")


    # --- 3. 특정 체크포인트의 이미지 그리드 시각화 및 저장 ---

    if steps_to_evaluate:
        # 가장 마지막 체크포인트의 이미지 그리드를 시각화하고 저장합니다.
        latest_step = steps_to_evaluate[-1]
        print(f"\n--- 최신 체크포인트 ({latest_step}) 이미지 그리드 시각화 및 저장 ---")
        try:
            G, F, E = load_model(config, device, latest_step)
            visualize_samples(config, device, G, F, latest_step)
        except Exception as e:
            print(f"이미지 시각화 오류: {e}")


if __name__ == '__main__':
    main()

Writing evaluate.py


In [113]:
!python evaluate.py

--- 프로젝트: FashionGAN_v1-9 ---
평가할 체크포인트 (5000 iter 간격): [5000, 10000, 15000, 20000]

[Evaluating Iteration 5000]
FID 계산을 시작합니다. (생성 이미지 1000개 디스크 저장 중...)
FID 계산을 시작합니다. (Inception V3 모델 로딩 중...)
/usr/local/lib/python3.12/dist-packages/torch_fidelity/datasets.py:16: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes())).view(height, width, 3)
FID Score @ 5000: 168.1529

[Evaluating Iteration 10000]
FID 계산을 시작합니다. (생성 이미지 1000개 디스크 저장 중...)
FID 계산을 시작합니다. (Inception V3 모델 로딩 중...)
FID Score @ 10000: 121.9674

[Evaluating Iteration 15000]
FID 계산을 시작합니다. (생성 이미지 1000개 디스크 저장 중...)
FID 계산을 시작합니다. (Inception V3 모델 로딩 중...)
FID Score @ 15000: 116.6672

[Evaluating Iteration 20000]
FID 계산을 시작합니다. 